# Interactive visualization of spatial single-cell data using Vitessce

This notebook explains how to create interactive visualizations of data that is accessible locally.


We progress through different visualization tasks, first demonstrating how Vitessce facilitates integrated imaging and spatial single-cell visualizations, then demonstrating visualization of non-spatial and image-only datasets.

<!--## Table of Contents

- SpatialData (via Zarr)
- AnnData via Zarr
- AnnData via H5AD
- OME-Zarr
- OME-TIFF
-->

## Utility dependencies

First, we import utility dependencies which will be used to download the example dataset and manipulate file paths, zip files, and JSON files.

In [ ]:
import os
from os.path import join, isfile, isdir
from urllib.request import urlretrieve
import zipfile
import json

## Dependencies for Vitessce

Here, we import classes and functions from the `vitessce` Python package.
This package includes not only APIs for [visualization configuration](https://python-docs.vitessce.io/api_config.html) but also [helper functions](https://python-docs.vitessce.io/api_data.html#vitessce-data-utils) for basic data transformation tasks.
To specify mappings between data fields and visualization properties, the package contains [classes](https://python-docs.vitessce.io/api_data.html#module-vitessce.wrappers) which wrap standard single-cell data structures stored in formats including [AnnData](https://doi.org/10.1101/2021.12.16.473007), [SpatialData](https://doi.org/10.1038/s41592-024-02212-x), [OME-TIFF](https://doi.org/10.1007/978-3-030-23937-4_1), and [OME-Zarr](https://doi.org/10.1038/s41592-021-01326-w):

- [AnnDataWrapper](https://python-docs.vitessce.io/api_data.html#vitessce.wrappers.AnnDataWrapper)
- [ImageOmeTiffWrapper](https://python-docs.vitessce.io/api_data.html#vitessce.wrappers.ImageOmeTiffWrapper)
- [ImageOmeZarrWrapper](https://python-docs.vitessce.io/api_data.html#vitessce.wrappers.ImageOmeZarrWrapper)
- [ObsSegmentationsOmeTiffWrapper](https://python-docs.vitessce.io/api_data.html#vitessce.wrappers.ObsSegmentationsOmeTiffWrapper)
- [ObsSegmentationsOmeZarrWrapper](https://python-docs.vitessce.io/api_data.html#vitessce.wrappers.ObsSegmentationsOmeZarrWrapper)
- [SpatialDataWrapper](https://python-docs.vitessce.io/api_data.html#vitessce.wrappers.SpatialDataWrapper)

In [ ]:
from vitessce import (
    VitessceConfig,
    ViewType as vt,
    CoordinationType as ct,
    CoordinationLevel as CL,
    SpatialDataWrapper,
    AnnDataWrapper,
    ImageOmeTiffWrapper,
    ImageOmeZarrWrapper,
    ObsSegmentationsOmeZarrWrapper,
    get_initial_coordination_scope_prefix,
    hconcat,
    vconcat,
)
from vitessce.data_utils import (
    VAR_CHUNK_SIZE,
    generate_h5ad_ref_spec,
    multiplex_img_to_ome_tiff,
    multiplex_img_to_ome_zarr,
)

## Dependencies for data structures

In this blog post, we perform basic data transformation tasks to save individual elements of an integrated SpatialData object to separate files in AnnData, OME-TIFF, and OME-Zarr formats.
To perform these data transformations, we import the following dependencies.
In general, you will typically not need to import all of these dependencies, either because you are only working with data in one of these formats, or because the data you intend to visualize is already saved to a file or directory.

Note: Dependencies such as `spatialdata` may need to be installed before they can be imported in the next code cell.

In [ ]:
import numpy as np
from spatialdata import read_zarr
from anndata import AnnData
from ome_zarr.writer import write_image
import tifffile
from generate_tiff_offsets import get_offsets

## Download example dataset

We download a mouse liver dataset which serves as a SpatialData [example dataset](https://github.com/scverse/spatialdata-notebooks/blob/main/notebooks/examples/transformations.ipynb).

This dataset was generated by [Guilliams et al.](https://doi.org/10.1016/j.cell.2021.12.018) and processed using [SPArrOW](https://doi.org/10.1101/2024.07.04.601829) during the SpatialData [developer workshop](https://doi.org/10.37044/osf.io/8ck3e) in 2024.

In [ ]:
data_dir = "data"
zip_filepath = join(data_dir, "mouse_liver.spatialdata.zarr.zip")
spatialdata_filepath = join(data_dir, "mouse_liver.spatialdata.zarr")
adata_zarr_filepath = join(data_dir, "mouse_liver.anndata.zarr")
adata_h5ad_filepath = join(data_dir, "mouse_liver.h5ad")
ref_spec_json_filepath = join(data_dir, "mouse_liver.h5ad.ref.json")
ome_tiff_filepath = join(data_dir, "mouse_liver.ome.tif")
offsets_json_filepath = join(data_dir, "mouse_liver.ome.tif.offsets.json")
ome_zarr_filepath = join(data_dir, "mouse_liver.ome.zarr")
labels_ome_zarr_filepath = join(data_dir, "mouse_liver.labels.ome.zarr")

The following code uses Python's `urlretrieve` to download the SpatialData object as a zip file, then unzips the file using the `zipfile` module.

In [ ]:
if not isdir(spatialdata_filepath):
    if not isfile(zip_filepath):
        os.makedirs(data_dir, exist_ok=True)
        urlretrieve('https://s3.embl.de/spatialdata/spatialdata-sandbox/mouse_liver.zip', zip_filepath)
    with zipfile.ZipFile(zip_filepath,"r") as zip_ref:
        zip_ref.extractall(data_dir)
        os.rename(join(data_dir, "data.zarr"), spatialdata_filepath)

# Visualization of a SpatialData object

SpatialData objects are the most complex type of data structure we will work with in this blog post.
SpatialData objects function as contains for multiple types of Spatial Elements:

- Tables (each table is represented as an AnnData object)
- Points (e.g., coordinates of transcripts from FISH-based experiments)
- Shapes (vector-based shapes such as polygons and circles)
- Labels (label images, i.e., segmentation bitmasks; each label image is stored using OME-Zarr)
- Images (microscopy images; each image is stored using OME-Zarr)

## Configure Vitessce

Vitessce needs to know which pieces of data we are interested in visualizing, the visualization types we would like to use, and how we want to coordinate (or link) the views.
To visualize data stored in a SpatialData object, we use the `SpatialDataWrapper` class and specify the paths (relative to the root of the Zarr [directory store](https://zarr.readthedocs.io/en/v2.18.5/api/storage.html#zarr.storage.DirectoryStore)) to different spatial elements of interest.

In [ ]:
# Create a VitessceConfig instance.
vc = VitessceConfig(schema_version="1.0.17", name="SpatialData Demo")

# Instantiate the wrapper class, specifying data fields of interest.
wrapper = SpatialDataWrapper(
    sdata_path=spatialdata_filepath,
    # The following paths are relative to the root of the SpatialData Zarr store on-disk.
    table_path="tables/table",
    image_path="images/raw_image",
    labels_path="labels/segmentation_mask",
    obs_feature_matrix_path="tables/table/X",
    obs_set_paths=["tables/table/obs/annotation"],
    obs_set_names=["Annotation"],
    region="nucleus_boundaries",
    coordinate_system="global",
    coordination_values={
      "obsType": "cell"   
    }
)
# Add a new dataset to the Vitessce configuration,
# then add the wrapper class instance to this dataset.
dataset = vc.add_dataset(name='Mouse Liver').add_object(wrapper)

# Add views (visualizations) to the configuration.
spatial = vc.add_view("spatialBeta", dataset=dataset)
feature_list = vc.add_view("featureList", dataset=dataset)
layer_controller = vc.add_view("layerControllerBeta", dataset=dataset)
obs_sets = vc.add_view("obsSets", dataset=dataset)
heatmap = vc.add_view("heatmap", dataset=dataset)

vc.link_views_by_dict([spatial, layer_controller], {
    "imageLayer": CL([{
        "photometricInterpretation": "BlackIsZero",
        "imageChannel": CL([{
            "spatialTargetC": 0,
            "spatialChannelColor": [255, 255, 255],
            "spatialChannelWindow": [0, 4000],
        }])
    }]),
}, scope_prefix=get_initial_coordination_scope_prefix("A", "image"))

vc.link_views_by_dict([spatial, layer_controller], {
    "segmentationLayer": CL([{
        "segmentationChannel": CL([{
            "obsColorEncoding": "cellSetSelection",
        }]),
    }]),
}, scope_prefix=get_initial_coordination_scope_prefix("A", "obsSegmentations"))

vc.link_views([spatial, layer_controller, feature_list, obs_sets, heatmap], ["obsType"], [wrapper.obs_type_label])

# Layout the views in a grid arrangement.
vc.layout((spatial / heatmap) | (layer_controller / (feature_list | obs_sets)));

### Render the widget

In [ ]:
vw = vc.widget()
vw

## Extract AnnData object from SpatialData object

The above example demonstrates how to visualize a spatial 'omics dataset containing not only single-cell information (e.g., a cell-by-gene expression matrix, cell type annotations) but also an image and cell segmentations.
To demonstrate how to use Vitessce to visualize data from a (non-spatial) single-cell experiment, we will extract this information from the SpatialData object and save it to a simpler [AnnData](https://anndata.readthedocs.io/) object (ignoring the imaging and spatially-resolved elements).

In [ ]:
sdata = read_zarr(spatialdata_filepath)
sdata

In [ ]:
adata = sdata.tables['table']
adata

As Zarr-formatted data can be easily visualized by Vitessce, we recommend saving the AnnData object to a Zarr store using the [write_zarr](https://anndata.readthedocs.io/en/stable/generated/anndata.AnnData.write_zarr.html) method.
Optionally, the shape of array chunks (for the AnnData `X` array) can be specified as a parameter, to optimize performance based on data access patterns.
For example, a common pattern is to visualize data across all cells for one gene.
To support such a pattern, the chunk shape can be specified as follows, `(total number of cells, small number of genes)`, resulting in tall-and-skinny array chunks.

In [ ]:
adata.write_zarr(adata_zarr_filepath, chunks=(adata.shape[0], VAR_CHUNK_SIZE))

Alternatively, your AnnData object may already be stored using the H5AD (HDF5-based) format.
To demonstrate this scenario, we save the object using the [write_h5ad](https://anndata.readthedocs.io/en/stable/generated/anndata.AnnData.write_h5ad.html) method.

In [ ]:
adata.write_h5ad(adata_h5ad_filepath)

To read H5AD-formatted data, Vitessce requires an accompanying JSON [references specification](https://fsspec.github.io/kerchunk/spec.html) file, which can be constructed using the `generate_h5ad_ref_spec` utility function.

In [ ]:
ref_dict = generate_h5ad_ref_spec(adata_h5ad_filepath)
with open(ref_spec_json_filepath, "w") as f:
    json.dump(ref_dict, f)

# Visualization of an AnnData object

### Zarr-based AnnData

In [ ]:
vc = VitessceConfig(schema_version="1.0.17", name="AnnData (zarr)")
# Add data.
wrapper = AnnDataWrapper(
    adata_path=adata_zarr_filepath,
    obs_feature_matrix_path="X",
    obs_set_paths=["obs/annotation"],
    obs_set_names=["Annotation"],
    coordination_values={
      "obsType": "cell"   
    }
)
dataset = vc.add_dataset(name='Mouse Liver').add_object(wrapper)

# Add views.
heatmap = vc.add_view(vt.HEATMAP, dataset=dataset)
feature_list = vc.add_view(vt.FEATURE_LIST, dataset=dataset)
obs_sets = vc.add_view(vt.OBS_SETS, dataset=dataset)
violin_plots = vc.add_view("obsSetFeatureValueDistribution", dataset=dataset)

vc.link_views([heatmap, feature_list, obs_sets], ['obsType', 'featureValueColormapRange'], ['cell', [0, 0.01]])

# Layout the views.
vc.layout((heatmap / violin_plots) | (feature_list / obs_sets));

In [ ]:
vc.widget()

### H5AD-based AnnData

In [ ]:
vc = VitessceConfig(schema_version="1.0.17", name="AnnData (h5ad)")
# Add data.
wrapper = AnnDataWrapper(
    adata_path=adata_h5ad_filepath,
    ref_path=ref_spec_json_filepath,
    obs_feature_matrix_path="X",
    obs_set_paths=["obs/annotation"],
    obs_set_names=["Annotation"],
    coordination_values={
      "obsType": "cell"   
    }
)
dataset = vc.add_dataset(name='Mouse Liver').add_object(wrapper)

# Add views.
heatmap = vc.add_view(vt.HEATMAP, dataset=dataset)
feature_list = vc.add_view(vt.FEATURE_LIST, dataset=dataset)
obs_sets = vc.add_view(vt.OBS_SETS, dataset=dataset)
violin_plots = vc.add_view("obsSetFeatureValueDistribution", dataset=dataset)

vc.link_views([heatmap, feature_list, obs_sets], ['obsType', 'featureValueColormapRange'], ['cell', [0, 0.01]])

# Layout the views.
vc.layout((heatmap / violin_plots) | (feature_list / obs_sets));

In [ ]:
vc.widget()

## Extract image from SpatialData object

In contrast to extraction of the non-spatial data from the SpatialData object, we can extract the imaging (and segmentation/label image) data and save it to a dedicated bioimaging file format.

In [ ]:
img_arr = sdata.images['raw_image'].to_numpy()
labels_arr = sdata.labels['segmentation_mask'].to_numpy()
labels_arr = labels_arr[np.newaxis, :]

### Save image to OME-Zarr

For small images, the data can be saved to OME-Zarr or OME-TIFF format using [utility functions](https://python-docs.vitessce.io/api_data.html#vitessce-data-utils) from the Vitessce package.
Larger images require generation of an [image pyramid](https://en.wikipedia.org/wiki/Pyramid_(image_processing)), which can be performed using tools from the OME ecosystem such as [bioformats2raw](https://github.com/glencoesoftware/bioformats2raw) and [raw2ometiff](https://github.com/glencoesoftware/raw2ometiff).

In [ ]:
multiplex_img_to_ome_zarr(img_arr, ["Channel 0"], ome_zarr_filepath)
multiplex_img_to_ome_zarr(labels_arr, ["cell"], labels_ome_zarr_filepath)

### Save image and segmentations to OME-TIFF

To efficiently visualize OME-TIFF data using Vitessce, a JSON-based offsets file can be constructed using [generate-tiff-offsets](https://github.com/hms-dbmi/generate-tiff-offsets).
This JSON file contains byte offsets into different partitions of the TIFF file, effectively resulting in an "indexed TIFF" which is described by [Manz et al. 2022](https://doi.org/10.1038/s41592-022-01482-7).

In [ ]:
multiplex_img_to_ome_tiff(img_arr, ["Channel 0"], ome_tiff_filepath)
offsets = get_offsets(ome_tiff_filepath)
with open(offsets_json_filepath, "w") as f:
    json.dump(offsets, f)

# Visualization of an image file

### OME-Zarr image

In [ ]:
vc = VitessceConfig(schema_version="1.0.17", name="Image (ome-zarr)")

# Add data.
img_wrapper = ImageOmeZarrWrapper(
    img_path=ome_zarr_filepath,
    coordination_values={
        "fileUid": "image",
    }
)
segmentations_wrapper = ObsSegmentationsOmeZarrWrapper(
    img_path=labels_ome_zarr_filepath,
    coordination_values={
        "fileUid": "segmentations",
    }
)
# Here, we chain .add_object calls to add both the image and segmentation
# wrapper instances to the same dataset.
dataset = vc.add_dataset(name='Mouse Liver').add_object(img_wrapper).add_object(segmentations_wrapper)

# Add views.
spatial = vc.add_view("spatialBeta", dataset=dataset)
layer_controller = vc.add_view("layerControllerBeta", dataset=dataset)

vc.link_views_by_dict([spatial, layer_controller], {
    'imageLayer': CL([{
        # In case there are multiple image files in our dataset,
        # we use fileUid to specify which file we intend to visualize
        # in this image layer.
        'fileUid': 'image',
        'photometricInterpretation': 'BlackIsZero',
        'imageChannel': CL([{
            'spatialTargetC': 0,
            'spatialChannelColor': [255, 255, 255],
            'spatialChannelWindow': [0, 4000],
        }])
    }]),
}, scope_prefix=get_initial_coordination_scope_prefix("A", "image"))

vc.link_views_by_dict([spatial, layer_controller], {
    'segmentationLayer': CL([{
        # In case there are multiple segmentation files in our dataset,
        # we use fileUid to specify which file we intend to visualize
        # in this segmentation layer.
        'fileUid': 'segmentations',
        'segmentationChannel': CL([{
            'obsColorEncoding': 'spatialChannelColor',
            'spatialChannelColor': [0, 255, 0],
            'spatialChannelOpacity': 0.75,
            'spatialSegmentationFilled': False,
            'spatialSegmentationStrokeWidth': 0.25,
        }]),
    }]),
}, scope_prefix=get_initial_coordination_scope_prefix("A", "obsSegmentations"))

vc.link_views([spatial, layer_controller, feature_list, obs_sets], ['obsType'], ['cell'])

# Layout the views.
vc.layout(hconcat(spatial, layer_controller, split=(2, 1)));

In [ ]:
vw = vc.widget()
vw

### OME-TIFF image

In [ ]:
vc = VitessceConfig(schema_version="1.0.17", name="Image and segmentations (ome-tiff)")
# Add data.
wrapper = ImageOmeTiffWrapper(
    img_path=ome_tiff_filepath,
    offsets_path=offsets_json_filepath
)
dataset = vc.add_dataset(name='Mouse Liver').add_object(wrapper)

# Add views.
spatial = vc.add_view("spatialBeta", dataset=dataset)
layer_controller = vc.add_view("layerControllerBeta", dataset=dataset)

vc.link_views_by_dict([spatial, layer_controller], {
    'imageLayer': CL([{
        'photometricInterpretation': 'BlackIsZero',
        'imageChannel': CL([{
            'spatialTargetC': 0,
            'spatialChannelColor': [255, 255, 255],
            'spatialChannelWindow': [0, 4000],
        }])
    }]),
}, scope_prefix=get_initial_coordination_scope_prefix("A", "image"))

# Layout the views.
vc.layout(hconcat(spatial, layer_controller, split=(2, 1)));

In [ ]:
vc.widget()

## Data location options

Vitessce can visualize data [not only stored](https://python-docs.vitessce.io/data_options.html) locally (referenced using local file or directory paths) but also stored remotely (referenced using absolute URL paths).
Depending on whether the Python kernel running the Jupyter process is running locally versus remotely (e.g., on a cluster or cloud platform such as Google Colab), certain data locations may be challenging to access from the machine running the Jupyter notebook frontend (i.e., the machine on which the web browser used to view the notebook is installed).

To provide data via one of these alternative mechanisms, use parameters with the following suffices when instantiating the data wrapper classes.

- `_path`: Local file or directory
- `_url`: Remote file or directory
- `_store`: Zarr-store-accessible (for zarr-based formats)
- `_artifact`: Lamin artifact

For example, `adata_path` can be exchanged for one of the following options.

```diff
AnnDataWrapper(
-    adata_path="./mouse_liver.spatialdata.zarr",
+    adata_url="https://example.com/mouse_liver.spatialdata.zarr", # Absolute URL
+    adata_store="./mouse_liver.spatialdata.zarr", # String interpreted as root of DirectoryStore
+    adata_store=zarr.DirectoryStore("./mouse_liver.spatialdata.zarr"), # Instance of zarr.storage
+    adata_artifact=adata_zarr_artifact, # Instance of ln.Artifact
    ...
```

Note that the `_store` options are only available for Zarr-based formats, such as AnnDataWrapper, SpatialDataWrapper, and ImageOmeZarrWrapper.
Further, when multiple files are required, such as both `adata_path` and `ref_path` (for the JSON reference specification file accompanying an H5AD file) or `img_path` and `offsets_path`, multiple parameter suffices may need to be changed.